# EDA
___

In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../../Data/Movies/IMDB_Movies_cleaned.csv')
df.head()

,title,year,genre,duration,country,language,director,writer,production_company,actors,avg_vote,votes,budget,usa_gross_income,worldwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,Metropolis,1927,"Drama, Sci-Fi",153,Germany,German,Fritz Lang,"Thea von Harbou, Thea von Harbou",Universum Film (UFA),"Alfred Abel, Gustav Fröhlich, Rudolf Klein-Rog...",8.3,148396,780000,1236166,1349711,98.0,471.0,194.0
1,City Lights,1931,"Comedy, Drama, Romance",87,USA,English,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Virginia Cherrill, Florence Lee, Harry Myers, ...",8.5,152716,1500000,19181,32609,99.0,270.0,120.0
2,Modern Times,1936,"Comedy, Drama, Family",87,USA,English,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Charles Chaplin, Paulette Goddard, Henry Bergm...",8.5,197969,1500000,163577,445226,96.0,262.0,146.0
3,Snow White and the Seven Dwarfs,1937,"Animation, Family, Fantasy",83,USA,English,"William Cottrell, David Hand","Jacob Grimm, Wilhelm Grimm",Walt Disney Productions,"Roy Atwell, Stuart Buchanan, Adriana Caselotti...",7.6,168735,1499000,184925486,184925486,95.0,229.0,166.0
4,Gone with the Wind,1939,"Drama, History, Romance",238,USA,English,"Victor Fleming, George Cukor","Margaret Mitchell, Sidney Howard",Selznick International Pictures,"Thomas Mitchell, Barbara O'Neil, Vivien Leigh,...",8.1,269664,3977000,200852579,402352579,97.0,808.0,183.0


In [3]:
df.describe()

,year,duration,avg_vote,votes,budget,usa_gross_income,worldwide_gross_income,metascore,reviews_from_users,reviews_from_critics
count,6459.000000,6459.000000,6459.000000,6.459000e+03,6.459000e+03,6.459000e+03,6.459000e+03,6459.000000,6459.000000,6459.000000
mean,2003.381174,107.951231,6.444682,9.354221e+04,3.048041e+07,4.097111e+07,8.829865e+07,54.955411,297.004800,147.188574
std,11.587292,18.858011,0.986718,1.606785e+05,3.985312e+07,6.704754e+07,1.743236e+08,18.092700,451.897507,124.048523
min,1927.000000,63.000000,1.400000,1.000000e+02,2.000000e+01,9.500000e+01,9.500000e+01,1.000000,1.000000,1.000000
25%,1999.000000,95.000000,5.900000,1.141400e+04,5.749889e+06,1.548588e+06,4.631308e+06,42.000000,74.000000,58.000000
50%,2006.000000,105.000000,6.500000,3.749300e+04,1.600000e+07,1.733637e+07,2.685746e+07,55.000000,164.000000,113.000000
75%,2012.000000,118.000000,7.100000,1.036910e+05,3.800000e+07,5.156309e+07,9.209381e+07,68.000000,339.000000,197.000000
max,2019.000000,272.000000,9.300000,2.159628e+06,3.560000e+08,9.366622e+08,2.797801e+09,100.000000,8302.000000,987.000000


In [4]:
df.loc[df['budget'] <= 200]

,title,year,genre,duration,country,language,director,writer,production_company,actors,avg_vote,votes,budget,usa_gross_income,worldwide_gross_income,metascore,reviews_from_users,reviews_from_critics
363,Poltergeist,1982,"Horror, Thriller",114,USA,English,Tobe Hooper,"Steven Spielberg, Michael Grais",Metro-Goldwyn-Mayer (MGM),"Craig T. Nelson, JoBeth Williams, Beatrice Str...",7.3,132785,20,77142388,77142388,79.0,375.0,250.0
6432,Grass,2018,Drama,66,South Korea,Korean,Sang-soo Hong,Sang-soo Hong,Jeonwonsa Film,"Min-hee Kim, Joobong Kee, Jae-hong Ahn, Jin-yo...",6.8,643,82,8084,53503,77.0,1.0,30.0


In [5]:
# Budget for "Pooltergeist" is actually $10.7 (M) - https://www.imdb.com/title/tt0084516/?ref_=nv_sr_srsg_0
# Budfet for "Grass" is technically correct - https://www.imdb.com/title/tt7913172/?ref_=fn_al_tt_1
df.loc[363, 'budget'] = 10_700_000

df.describe()

,year,duration,avg_vote,votes,budget,usa_gross_income,worldwide_gross_income,metascore,reviews_from_users,reviews_from_critics
count,6459.000000,6459.000000,6459.000000,6.459000e+03,6.459000e+03,6.459000e+03,6.459000e+03,6459.000000,6459.000000,6459.000000
mean,2003.381174,107.951231,6.444682,9.354221e+04,3.048207e+07,4.097111e+07,8.829865e+07,54.955411,297.004800,147.188574
std,11.587292,18.858011,0.986718,1.606785e+05,3.985207e+07,6.704754e+07,1.743236e+08,18.092700,451.897507,124.048523
min,1927.000000,63.000000,1.400000,1.000000e+02,8.200000e+01,9.500000e+01,9.500000e+01,1.000000,1.000000,1.000000
25%,1999.000000,95.000000,5.900000,1.141400e+04,5.768389e+06,1.548588e+06,4.631308e+06,42.000000,74.000000,58.000000
50%,2006.000000,105.000000,6.500000,3.749300e+04,1.600000e+07,1.733637e+07,2.685746e+07,55.000000,164.000000,113.000000
75%,2012.000000,118.000000,7.100000,1.036910e+05,3.800000e+07,5.156309e+07,9.209381e+07,68.000000,339.000000,197.000000
max,2019.000000,272.000000,9.300000,2.159628e+06,3.560000e+08,9.366622e+08,2.797801e+09,100.000000,8302.000000,987.000000


In [6]:
df.loc[df['usa_gross_income'] <= 95]

,title,year,genre,duration,country,language,director,writer,production_company,actors,avg_vote,votes,budget,usa_gross_income,worldwide_gross_income,metascore,reviews_from_users,reviews_from_critics
4104,The Objective,2008,"Horror, Sci-Fi, Thriller",90,"USA, Morocco","English, Pushto",Daniel Myrick,"Daniel Myrick, Mark A. Patton",Jaz Films,"Jonas Ball, Matthew R. Anderson, Jon Huertas, ...",5.5,7634,4000000,95,95,26.0,67.0,37.0


In [7]:
# according to outside research (IMDB, BoxOfficMojo, RottenTomatoes) the gross is correct, however it's a heavy outlier 
# and is more than likely an error, so it will be droped
df.drop(index=4104, inplace=True)
df.reset_index(drop=True, inplace=True)
df.describe()

,year,duration,avg_vote,votes,budget,usa_gross_income,worldwide_gross_income,metascore,reviews_from_users,reviews_from_critics
count,6458.000000,6458.000000,6458.000000,6.458000e+03,6.458000e+03,6.458000e+03,6.458000e+03,6458.000000,6458.000000,6458.000000
mean,2003.380458,107.954011,6.444828,9.355551e+04,3.048617e+07,4.097745e+07,8.831232e+07,54.959895,297.040415,147.205636
std,11.588047,18.858148,0.986724,1.606874e+05,3.985380e+07,6.705080e+07,1.743337e+08,18.090512,451.923433,124.050548
min,1927.000000,63.000000,1.400000,1.000000e+02,8.200000e+01,4.230000e+02,4.230000e+02,1.000000,1.000000,1.000000
25%,1999.000000,95.000000,5.900000,1.142525e+04,5.777000e+06,1.556678e+06,4.636574e+06,42.000000,74.250000,58.000000
50%,2006.000000,105.000000,6.500000,3.749450e+04,1.600000e+07,1.733998e+07,2.687465e+07,55.000000,164.000000,113.000000
75%,2012.000000,118.000000,7.100000,1.037115e+05,3.800000e+07,5.157166e+07,9.213769e+07,68.000000,339.000000,197.000000
max,2019.000000,272.000000,9.300000,2.159628e+06,3.560000e+08,9.366622e+08,2.797801e+09,100.000000,8302.000000,987.000000


In [8]:
df.loc[df['worldwide_gross_income'] <= 500]

,title,year,genre,duration,country,language,director,writer,production_company,actors,avg_vote,votes,budget,usa_gross_income,worldwide_gross_income,metascore,reviews_from_users,reviews_from_critics
3164,The Dark Hours,2005,"Horror, Thriller",80,Canada,English,Paul Fox,Wil Zmak,Calder Road Films,"Kate Greenhouse, Bruce McFee, Jeff Seymour, Da...",6.0,5165,355000,423,423,63.0,51.0,78.0


In [9]:
# this also appears to be an error so it'll be droped
df.drop(index=3164, inplace=True)
df.reset_index(drop=True, inplace=True)
df.describe()

,year,duration,avg_vote,votes,budget,usa_gross_income,worldwide_gross_income,metascore,reviews_from_users,reviews_from_critics
count,6457.000000,6457.000000,6457.000000,6.457000e+03,6.457000e+03,6.457000e+03,6.457000e+03,6457.000000,6457.000000,6457.000000
mean,2003.380208,107.958340,6.444897,9.356920e+04,3.049084e+07,4.098380e+07,8.832600e+07,54.958650,297.078519,147.216354
std,11.588927,18.856399,0.986785,1.606961e+05,3.985512e+07,6.705405e+07,1.743437e+08,18.091636,451.948057,124.057165
min,1927.000000,63.000000,1.400000,1.000000e+02,8.200000e+01,5.090000e+02,5.940000e+02,1.000000,1.000000,1.000000
25%,1999.000000,95.000000,5.900000,1.144100e+04,5.777000e+06,1.558836e+06,4.644108e+06,42.000000,75.000000,58.000000
50%,2006.000000,105.000000,6.500000,3.749600e+04,1.600000e+07,1.734360e+07,2.689185e+07,55.000000,164.000000,113.000000
75%,2012.000000,118.000000,7.100000,1.037320e+05,3.800000e+07,5.158024e+07,9.218157e+07,68.000000,339.000000,197.000000
max,2019.000000,272.000000,9.300000,2.159628e+06,3.560000e+08,9.366622e+08,2.797801e+09,100.000000,8302.000000,987.000000


In [10]:
df.loc[df['worldwide_gross_income'] <= 1_000]

,title,year,genre,duration,country,language,director,writer,production_company,actors,avg_vote,votes,budget,usa_gross_income,worldwide_gross_income,metascore,reviews_from_users,reviews_from_critics
181,The Last Wave,1977,"Drama, Fantasy, Mystery",106,Australia,"English, Italian, Aboriginal",Peter Weir,"Peter Weir, Tony Morphett",McElroy & McElroy,"Richard Chamberlain, Olivia Hamnett, David Gul...",7.1,7959,518400,957,957,85.0,52.0,68.0
2329,The Jimmy Show,2001,"Comedy, Drama",96,USA,English,Frank Whaley,"Jonathan Marc Sherman, Frank Whaley",Next Wednesday Productions,"Frank Whaley, Carla Gugino, Ethan Hawke, Lynn ...",5.2,514,1500000,1000,1000,39.0,20.0,9.0
3147,Steve + Sky,2004,"Romance, Drama, Comedy",97,Belgium,Dutch,Felix van Groeningen,Felix van Groeningen,Favourite Films NV,"Titus De Voogdt, Delfine Bafort, Johan Heldenb...",6.7,1000,817500,624,624,44.0,4.0,12.0
3642,Shake Hands with the Devil,2007,"Drama, History, War",112,Canada,"English, French",Roger Spottiswoode,"Roméo Dallaire, Michael Donovan",Barna-Alper Productions,"Roy Dupuis, Owen Sejake, James Gallanders, Mic...",7.6,3084,7810000,594,594,57.0,18.0,21.0
5123,Edwin Boyd,2011,"Crime, Drama",105,Canada,English,Nathan Morlando,Nathan Morlando,IFC Films,"Scott Speedman, Kelly Reilly, Kevin Durand, Jo...",6.2,4057,5000000,625,625,56.0,20.0,33.0


In [11]:
# "The Last Wave" worldwide gross is for a release and the original box office gross is not know, so drop it
# https://www.boxofficemojo.com/title/tt0076299/?ref_=bo_se_r_1

# due to inconsistancies "The Jimmy Show" will also be droped
# the rest are fine

df.drop(index=[181, 2329], inplace=True)
df.reset_index(drop=True, inplace=True)
df.describe()

,year,duration,avg_vote,votes,budget,usa_gross_income,worldwide_gross_income,metascore,reviews_from_users,reviews_from_critics
count,6455.000000,6455.000000,6455.000000,6.455000e+03,6.455000e+03,6.455000e+03,6.455000e+03,6455.000000,6455.000000,6455.000000
mean,2003.384663,107.960496,6.444988,9.359688e+04,3.049997e+07,4.099650e+07,8.835337e+07,54.956468,297.159411,147.250039
std,11.586031,18.858717,0.986782,1.607133e+05,3.985791e+07,6.706056e+07,1.743638e+08,18.089484,451.994617,124.060533
min,1927.000000,63.000000,1.400000,1.000000e+02,8.200000e+01,5.090000e+02,5.940000e+02,1.000000,1.000000,1.000000
25%,1999.000000,95.000000,5.900000,1.146200e+04,5.788500e+06,1.568678e+06,4.644498e+06,42.000000,75.000000,58.000000
50%,2006.000000,105.000000,6.500000,3.756400e+04,1.600000e+07,1.736802e+07,2.689548e+07,55.000000,164.000000,113.000000
75%,2012.000000,118.000000,7.100000,1.037355e+05,3.800000e+07,5.159012e+07,9.223271e+07,68.000000,339.500000,197.000000
max,2019.000000,272.000000,9.300000,2.159628e+06,3.560000e+08,9.366622e+08,2.797801e+09,100.000000,8302.000000,987.000000


___
## production_company dummy

In [12]:
df['production_company'].value_counts()

Universal Pictures                296
Warner Bros.                      291
Columbia Pictures                 284
Paramount Pictures                273
Twentieth Century Fox             207
                                 ... 
Samuels Media                       1
Best Little Boy Productions         1
Oldgarth Media                      1
3 Miles Apart Productions Ltd.      1
Hero Entertainment                  1
Name: production_company, Length: 2384, dtype: int64

In [13]:
# creating dummies for production_company
pc_dummy_df = pd.get_dummies(df['production_company']).add_prefix('prod_co-')
pc_dummy_df.rename(columns = lambda x: x.replace(' ', '_').lower(), inplace= True)
pc_dummy_df.head()

,"prod_co-""dia""_productions_gmbh_&_co._kg",prod_co-100%_synthetic_films,prod_co-10th_hole_productions,prod_co-120db_films,prod_co-1212_entertainment,prod_co-13_productions,prod_co-1492_pictures,prod_co-1818_productions,prod_co-1821_pictures,prod_co-19_entertainment,...,prod_co-zonbo_media,prod_co-zucker_brothers_productions,prod_co-double_a_films,prod_co-erbp,prod_co-here!_films,prod_co-i_am_other,prod_co-i5_films,prod_co-micro_scope,prod_co-nwave_pictures,prod_co-thefyzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


___
## genre dummy

In [14]:
genre_dummy_df = df['genre'].map(lambda x: x.split(', ')).str.join('|').str.get_dummies().add_prefix('genre-')
genre_dummy_df.rename(columns = lambda x: x.replace(' ', '_').lower(), inplace=True)
genre_dummy_df.head()

,genre-action,genre-adventure,genre-animation,genre-biography,genre-comedy,genre-crime,genre-drama,genre-family,genre-fantasy,genre-film-noir,...,genre-horror,genre-music,genre-musical,genre-mystery,genre-romance,genre-sci-fi,genre-sport,genre-thriller,genre-war,genre-western
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


___
## country dummy

In [15]:
country_dum_df = df['country'].map(lambda x: x.split(', ')).str.join('|').str.get_dummies().add_prefix('country-')
country_dum_df.rename(columns = lambda x: x.replace(' ', '_').lower(), inplace = True)
country_dum_df.head()

,country-afghanistan,country-algeria,country-angola,country-argentina,country-aruba,country-australia,country-austria,country-bahamas,country-belgium,country-bhutan,...,country-turkey,country-uk,country-usa,country-ukraine,country-united_arab_emirates,country-uruguay,country-venezuela,country-vietnam,country-west_germany,country-yugoslavia
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


___
## writer dummy

In [16]:
writer_dum_df = df['writer'].map(lambda x: x.split(', ')).str.join('|').str.get_dummies().add_prefix('writer-')
writer_dum_df.rename(columns = lambda x: x.replace(' ', '_').lower(), inplace = True)
writer_dum_df.head()

,writer-'a.j.'_marriot,writer-'weird_al'_yankovic,writer-a_l_katz,writer-a.a._milne,writer-a.e._hotchner,writer-a.e.w._mason,writer-a.j._quinnell,writer-a.s._byatt,writer-aaron_abrams,writer-aaron_covington,...,writer-zoë_lund,writer-àlex_pastor,writer-álex_de_la_iglesia,writer-álvaro_rodríguez,writer-élie_chouraqui,writer-émile_gaudreault,writer-émile_zola,writer-éric_rohmer,writer-éric_toledano,writer-ólafur_egilsson
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


___
## language dummy

In [17]:
lang_dum_df = df['language'].map(lambda x: x.replace(', ', '|')).str.get_dummies().add_prefix('language-')
lang_dum_df.rename(columns = lambda x: x.replace(' ', '_').lower(), inplace = True)
lang_dum_df.head()

,language-aboriginal,language-acholi,language-afrikaans,language-akan,language-albanian,language-algonquin,language-american_sign_language,language-amharic,language-ancient_(to_1453),language-apache_languages,...,language-urdu,language-uzbek,language-vietnamese,language-visayan,language-washoe,language-welsh,language-xhosa,language-yiddish,language-yoruba,language-zulu
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


___
## director dummy

In [18]:
dir_dum_df = df['director'].map(lambda x: x.replace(', ', '|')).str.get_dummies().add_prefix('director-')
dir_dum_df.rename(columns = lambda x: x.replace(' ', '_').lower(), inplace=True)
dir_dum_df.head()

,director-aaron_blaise,director-aaron_horvath,director-aaron_schneider,director-aaron_seltzer,director-aaron_sorkin,director-aaron_wilson,director-aaron_woodley,director-abbas_kiarostami,director-abby_kohn,director-abdellah_taia,...,director-zhangke_jia,director-ziad_doueiri,director-zoe_r._cassavetes,director-ágnes_hranitzky,director-álex_de_la_iglesia,director-élie_chouraqui,director-émile_gaudreault,director-éric_rohmer,director-éric_toledano,director-éric_warin
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


___
## actors dummy

In [ ]:
actor_dum_df = df['actors'].map(lambda x: x.replace(', ', '|')).str.get_dummies().add_prefix('actor-')
actor_dum_df.rename(columns = lambda x: x.replace(' ', '_').lower(), inplace=True)
actor_dum_df.head()

___

In [ ]:
df.head()